In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

/home/trec/anaconda3/envs/PyBulletRL/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
word_to_ix = {'hello':0 , 'world':1}
embeds = nn.Embedding(2,5)#2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix['hello']],dtype=torch.long)
print(lookup_tensor)

hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([0])
tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward0>)


An Example: N-Gram Language Modeling
Recall that in an n-gram language model, given a sequence of words 
https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html
​
  is the ith word of the sequence.

In this example, we will compute the loss function on some training examples and update the parameters with backpropagation.

In [4]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

# we should tokenize the input, but we will ignore that for now
# build a list of tuples.
# Each tuple is ([ word_i-CONTEXT_SIZE, ..., word_i-1 ], target word)

ngrams = [([test_sentence[i-j-1] for j in range(CONTEXT_SIZE)], test_sentence[i]) for i in range(CONTEXT_SIZE, len(test_sentence))]

#print the first three to see how they look like
print(ngrams[:3])

vocab = set(test_sentence)
print(vocab)
word_to_ix = {word: i for i, word in enumerate(vocab)}


        


[(['forty', 'When'], 'winters'), (['winters', 'forty'], 'shall'), (['shall', 'winters'], 'besiege')]
{'sum', 'an', 'succession', 'my', 'sunken', 'all-eating', 'so', 'thy', "totter'd", 'brow,', 'say,', 'in', 'deep', 'answer', 'much', 'To', 'use,', 'asked,', 'and', 'within', 'dig', 'praise', 'warm', 'forty', 'fair', 'see', "youth's", 'besiege', 'Shall', 'count,', 'Proving', 'thine', 'Then', "deserv'd", 'his', 'shall', 'days;', 'praise.', "'This", "beauty's", 'Thy', 'old,', 'winters', 'held:', 'the', 'worth', 'being', 'gazed', 'couldst', 'make', 'it', 'made', 'When', 'where', 'This', 'all', 'of', 'cold.', 'If', "feel'st", 'How', 'to', 'shame,', 'now,', 'Where', 'were', 'new', 'lusty', 'small', 'thine!', 'on', 'proud', 'Will', 'by', 'old', 'child', 'weed', 'a', 'when', 'field,', 'own', 'And', 'beauty', 'eyes,', 'more', 'Were', 'thriftless', 'trenches', 'thou', 'be', "excuse,'", 'mine', 'art', 'blood', 'treasure', 'livery', 'lies,'}


In [7]:
class NGramLanguageModeler(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, context_size) -> None:
        super(NGramLanguageModeler,self).__init__() 
        self.embeddings = nn.Embedding(vocab_size,embedding_dim)
        self.linear1 = nn.Linear(context_size*embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)
        
    def forward(self,inputs):
        embeds = self.embeddings(inputs).view((1,-1)) # adding a second dim to vector
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out,dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab),EMBEDDING_DIM,CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(),lr=0.001)

for epoch in range(1000):
    total_loss =0
    for context, target in ngrams:
        
        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in variables)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
        
        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()
        
        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)
        
        #step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a variable)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))
        
        #step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()
        
        #Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses) # The loss decreased every iteration over the training data!

# To get the embedding of a particular word, e.g. "beauty"
print(model.embeddings.weight[word_to_ix['beauty']])
        
        

[519.4787640571594, 517.0326201915741, 514.6028609275818, 512.1883540153503, 509.78805589675903, 507.4008240699768, 505.0261323451996, 502.6616702079773, 500.30630111694336, 497.9597191810608, 495.6220886707306, 493.29232954978943, 490.9706451892853, 488.6558322906494, 486.3483316898346, 484.0473816394806, 481.752153635025, 479.4602644443512, 477.1714360713959, 474.8845126628876, 472.59929394721985, 470.31683015823364, 468.0363202095032, 465.75568532943726, 463.47450947761536, 461.19114685058594, 458.9066233634949, 456.6196632385254, 454.33201575279236, 452.04023790359497, 449.7452201843262, 447.44635939598083, 445.14172887802124, 442.83183240890503, 440.5163514614105, 438.19497656822205, 435.86787128448486, 433.53356409072876, 431.1936423778534, 428.8464186191559, 426.49256896972656, 424.13024139404297, 421.7598783969879, 419.3804044723511, 416.9916594028473, 414.5930941104889, 412.18665277957916, 409.7707533836365, 407.34636294841766, 404.91226387023926, 402.4693182706833, 400.016586